# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Documentation](https://planetlabs.github.io/planet-client-python/index.html)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API.

The basic workflow for interacting with the Data API is:
1. search item types based on filters
1. activate assets
1. download assets

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [ ]:
from datetime import datetime
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter
import json
import requests
import time
import os

# We will also create a small helper function to print out JSON with proper indentation.
def p(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. 

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'

client = Auth.from_key(API_KEY)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

Let's also read in a GeoJSON geometry into a variable so we can use it during testing.

In [ ]:
with open("data/san-francisco.json") as f:
    geom = json.loads(f.read())

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.


In [ ]:
# Define the filters we'll use to find our data

item_types = ["REOrthoTile", "PSOrthoTile"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", datetime(month=1, day=1, year=2017))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [ ]:
p(combined_filter)

Now let's build the request:

In [ ]:
async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='planet_client_demo',search_filter=combined_filter, item_types=item_types)

In [ ]:
p(request)

In [ ]:
# Search the Data API
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.run_search(search_id=request['id'])
    item_list = [i async for i in items]

Now, we can iterate through our search results.

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

In [ ]:
with open('output/results.json','w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

This GeoJSON file can be opened and viewed in any compatible application.

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads.

The list of assets for an item that a user has access to can be retrieved with `permissions`

In [ ]:
print(item['id'])

In [ ]:
item['_permissions']

In [ ]:
# Get the assets link for the item
assets_url = item["_links"]["assets"]

# Print the assets link
print(assets_url)

In [ ]:
# Send a GET request to the assets url for the item (Get the list of available assets for the item)
res = session.get(assets_url)

# Assign a variable to the response
assets = res.json()

In [ ]:
# Assign a variable to the analytic asset from the item's assets
analytic_asset = assets["analytic"]

# Print the analytic asset data
p(analytic_asset)

In [ ]:
# Setup the activation url for a particular asset (in this case an analytic asset)
activation_url = analytic_asset["_links"]["activate"]

# Send a request to the activation url to activate the item
res = session.get(activation_url)

# Print the response from the activation request
p(res.status_code)

A response of 202 means that the request has been accepted and the activation will begin shortly. A 204 code indicates that the asset is already active and no further action is needed. A 401 code means the user does not have permissions to download this file.

Below, we are polling the API until the item is done activation. This may take awhile.

In [ ]:
asset_activated = False

while asset_activated == False:
    # Send a request to the item's assets url
    res = session.get(assets_url)

    # Assign a variable to the item's assets url response
    assets = res.json()

    asset_status = analytic_asset["status"]
    print(asset_status)
    time.sleep(15) 
    
    # If asset is already active, we are done
    if asset_status == 'active':
        asset_activated = True
        print("Asset is active and ready to download")

# Print the analytic asset data    
p(analytic_asset)

In [ ]:
callback = api.write_to_file(directory='output/')
body = client.download(assets['analytic_xml'], callback=callback)
print(body)

## Saved Searches

The Python API client can also help in managing saved searches on the Planet Platform.

In [ ]:
async with Session() as sess:
    cl = DataClient(sess)
    searches = await cl.list_searches()
    searches_list = [i async for i in searches]

In [ ]:
# We may have a lot of saved searches!

len(searches_list)

View your saved searches:

In [ ]:
for search in searches_list[:100]:
    print(search['id'], search['name'])

We can find the saved search we're looking for by filtering on our search id:

In [ ]:
async with Session() as sess:
    cl = DataClient(sess)
    search = await cl.get_search(searches_list[0]['id'])

In [ ]:
search

# Statistics

The Python API client can also help report statistical summaries of the amount of data in the Planet API.

In [ ]:
# Here, we can look at the stats for the search we've been working with:

async with Session() as sess:
    cl = DataClient(sess)
    stats = await cl.get_stats(interval='year', search_filter=combined_filter, item_types=item_types)

In [ ]:
p(stats)